In [59]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import os
from dbpd.preprocessing_config import PPGPreprocessingConfig, IMUPreprocessingConfig
from dbpd.ppg_preprocessing import scan_and_sync_segments, preprocess_ppg_data
from dbpd.heart_rate_analysis_config import HeartRateFeatureExtractionConfig
from dbpd.heart_rate_analysis import extract_signal_quality_features

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
# Cell has the tag 'parameters'
path_to_data =  '../../../tests/data'

quality_classifier_path = "/Users/peter/repos/dbpd/dbpd-toolbox2/src/dbpd/ppg/classifier/LR_PPG_quality.pkl"
path_to_sensor_data = os.path.join(path_to_data, '1.sensor_data')
path_to_preprocessed_data = os.path.join(path_to_data, '2.preprocessed_data', 'ppg')
path_to_quality_features = os.path.join(path_to_data, '3.extracted_features', 'ppg')

In [61]:
ppg_config = PPGPreprocessingConfig()
imu_config = IMUPreprocessingConfig()
metadatas_ppg, metadatas_imu = scan_and_sync_segments(os.path.join(path_to_sensor_data, 'ppg'),
                                                       os.path.join(path_to_sensor_data, 'imu'))
preprocess_ppg_data(metadatas_ppg[0], metadatas_imu[0],
                    os.path.join(path_to_preprocessed_data, 'ppg'),
                    ppg_config, imu_config)

Shape of the original data: (64775, 2) (72947, 4)
Shape of the overlapping segments: (64775, 2) (64361, 4)


In [65]:
config = HeartRateFeatureExtractionConfig()
extract_signal_quality_features(path_to_preprocessed_data, quality_classifier_path, path_to_quality_features, config)

[[ 0.51444303 82.        ]]
300 150 1000
(501,) (151,)


/Users/peter/Library/Caches/pypoetry/virtualenvs/dbpd-ZysVbjC7-py3.9/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 300 is greater than input length  = 3, using nperseg = 3
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


ValueError: operands could not be broadcast together with shapes (500,) (599,) 

In [11]:
import tsdf
import pandas as pd

metadata_dict = tsdf.load_metadata_from_path(os.path.join(output_path, 'PPG_meta.json'))
metadata_time = metadata_dict["PPG_time.bin"]
metadata_samples = metadata_dict["PPG_samples.bin"]
tested_data = tsdf.load_dataframe_from_binaries([metadata_time, metadata_samples], tsdf.constants.ConcatenationType.columns)

metadata_dict = tsdf.load_metadata_from_path(os.path.join(output_path, 'ref', 'PPG_meta.json'))
metadata_time = metadata_dict["PPG_time.bin"]
metadata_samples = metadata_dict["PPG_samples.bin"]
ref_data = tsdf.load_dataframe_from_binaries([metadata_time, metadata_samples], tsdf.constants.ConcatenationType.columns)

print("Comparing datasets: tested vs reference")
print("Shapes:", tested_data.shape, ref_data.shape)
desc_tested = tested_data.describe()
desc_ref = ref_data.describe()
summary = pd.concat([desc_tested, desc_ref], axis=1, keys=['Tested', 'Reference'])
print(summary)
print(tested_data.head())
print(ref_data.head())

Comparing datasets: tested vs reference
Shapes: (19338, 2) (19338, 2)
             Tested                   Reference              
               time         green          time         green
count  1.933800e+04  19338.000000  19338.000000  19338.000000
mean   1.624813e+09   3326.260086    322.283333      3.820214
std    1.860846e+02    390.998313    186.084803   5919.183045
min    1.624813e+09   2534.790484      0.000000 -47001.363761
25%    1.624813e+09   3045.056516    161.141667  -1099.888350
50%    1.624813e+09   3181.907103    322.283333    -24.494001
75%    1.624813e+09   3717.021484    483.425000   1159.509095
max    1.624813e+09   4323.557541    644.566667  48981.992338
           time        green
0  1.624813e+09  3048.661742
1  1.624813e+09  3027.368811
2  1.624813e+09  3014.800278
3  1.624813e+09  2992.760882
4  1.624813e+09  2975.891149
       time        green
0  0.000000 -1112.606896
1  0.033333 -3082.665426
2  0.066667 -4732.866602
3  0.100000 -5833.747274
4  0.133333